In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import time
import datetime

In [2]:
Location = r'C:\Users\Grueling\Downloads\coursera\features.csv'
features = pd.read_csv(Location, index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
Location_test = r'C:\Users\Grueling\Downloads\coursera\features_test.csv'
features_test = pd.read_csv(Location_test, index_col='match_id')

features_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


In [4]:
#Градиентный бустинг

In [5]:
#Определение столбцов которых нет в тестовой выборке
F = {features_test.columns[a]: a for a in range(102)}
for i in features.columns:
    if F.get(i) == None:
        print(i)

duration
radiant_win
tower_status_radiant
tower_status_dire
barracks_status_radiant
barracks_status_dire


In [6]:
#Запись целевой переменной
y = features['radiant_win']

In [7]:
#Удаление столбцов отсутсвующих в тестовой выборке
del features['duration']
del features['radiant_win']
del features['tower_status_radiant']
del features['tower_status_dire']
del features['barracks_status_radiant']
del features['barracks_status_dire']

In [8]:
#Определение столбцов с пропусками и количество пропусков
for i in features.columns:
    if features[i].count() < 97230:
        print(i, 97230 - features[i].count())

first_blood_time 19553
first_blood_team 19553
first_blood_player1 19553
first_blood_player2 43987
radiant_bottle_time 15691
radiant_courier_time 692
radiant_flying_courier_time 27479
radiant_first_ward_time 1836
dire_bottle_time 16143
dire_courier_time 676
dire_flying_courier_time 26098
dire_first_ward_time 1826


In [9]:
# first_blood_time и first_blood_team пропуски возможны по причине того, что
# в 19553 матчах не было за 5 минут first_blood, а наступил несколько позднее

In [10]:
#Заполнение пропусков нулями
features = features.fillna(0)

In [11]:
cv = KFold(5, shuffle=True, random_state=241)

In [12]:
i = 10
CVS = []
time = []
while i <= 30:
    start_time = datetime.datetime.now()

    clf = GradientBoostingClassifier(n_estimators=i, verbose=True, random_state=241, max_depth=3)
    
    CVS.append(np.mean(cross_val_score(estimator=clf, X=features, y=y, cv=cv, scoring='roc_auc')))
    time.append(datetime.datetime.now() - start_time)
    i += 10

      Iter       Train Loss   Remaining Time 
         1           1.3786           31.52s
         2           1.3729           28.19s
         3           1.3680           24.98s
         4           1.3636           21.15s
         5           1.3592           17.57s
         6           1.3546           13.89s
         7           1.3502           10.38s
         8           1.3461            6.95s
         9           1.3419            3.47s
        10           1.3381            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3781           35.38s
         2           1.3725           29.76s
         3           1.3673           26.40s
         4           1.3629           22.57s
         5           1.3581           18.71s
         6           1.3534           14.87s
         7           1.3487           11.60s
         8           1.3445            7.71s
         9           1.3408            3.83s
        10           1.3368            0.00s
      It

In [13]:
for i in range(3):
    print('CrossValidation')
    print(CVS[i])
    print('Time elapsed:')
    print(time[i])

CrossValidation
0.664387720635
Time elapsed:
0:03:26.922835
CrossValidation
0.682853573534
Time elapsed:
0:06:14.154401
CrossValidation
0.689496206059
Time elapsed:
0:08:42.782901


In [14]:
#При использовании 30 деревьев полученная точность 0.689496206059
# затраченное время 0:08:42.782901

In [15]:
clf.fit(features, y)

      Iter       Train Loss   Remaining Time 
         1           1.3786            2.44m
         2           1.3732            2.28m
         3           1.3681            2.15m
         4           1.3636            2.05m
         5           1.3589            1.96m
         6           1.3547            1.88m
         7           1.3502            1.79m
         8           1.3461            1.70m
         9           1.3422            1.61m
        10           1.3385            1.53m
        20           1.3092           45.78s
        30           1.2892            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=241,
              subsample=1.0, verbose=True, warm_start=False)

In [16]:
pred = clf.predict_proba(features)[:, 1]

In [17]:
print(roc_auc_score(y,pred))

0.70020888039


In [18]:
clf = GradientBoostingClassifier(n_estimators=40, verbose=True, random_state=241, max_depth=3)
    
CV = (np.mean(cross_val_score(estimator=clf, X=features, y=y, cv=cv, scoring='roc_auc')))
print(CV)

      Iter       Train Loss   Remaining Time 
         1           1.3786            3.43m
         2           1.3729            2.98m
         3           1.3680            2.69m
         4           1.3636            2.56m
         5           1.3592            2.41m
         6           1.3546            2.27m
         7           1.3502            2.15m
         8           1.3461            2.05m
         9           1.3419            1.96m
        10           1.3381            1.88m
        20           1.3090            1.19m
        30           1.2890           35.14s
        40           1.2745            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3781            2.29m
         2           1.3725            2.23m
         3           1.3673            2.16m
         4           1.3629            2.10m
         5           1.3581            2.02m
         6           1.3534            1.95m
         7           1.3487            1.90m
        

In [19]:
#Можно заметить улучшение точности при использовании деревьев более 30 (в примере используется 40)
# точность равна 0.694131121473

In [20]:


#Логистическая регрессия




In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
import warnings
warnings.simplefilter('ignore')

In [22]:
kf = KFold(5, shuffle=True, random_state=42)

In [23]:
scale = StandardScaler()
Xscl=scale.fit_transform(features) #Масштабируем

In [25]:
CVL = []
time_LR = []
for i in range(10):
    start_time = datetime.datetime.now()

    k = 10**(i-5)
    LR = LogisticRegression(C=k)
    CVL.append(np.mean(cross_val_score(estimator=LR, X=Xscl, y=y, cv=kf, scoring='roc_auc')))
    
    time_LR.append(datetime.datetime.now() - start_time)

In [26]:
C = 10**(CVL.index(max(CVL))-5)
T = time_LR[CVL.index(max(CVL))]
print('Score')
print(max(CVL))
print('C')
print(C)
print('time')
print(T)

Score
0.716550269726
C
0.01
time
0:01:00.981488


In [27]:
#Качество оценки немного лучше,чем у градиентного бустинга время затрачено меньше. 
#Лучший парметр регуляризации С=0.01

In [28]:
#Запись столбцов для расчета количества героев
X = features[ ['r1_hero', 'r2_hero','r3_hero','r4_hero','r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'] ].as_matrix()
X_hero = np.unique(X)
X_hero

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 109, 110, 112], dtype=int64)

In [29]:
# всего 113 героев, но не выбирают  героев 24, 107, 108, 111, 113 либо их просто нет 
# уникальных получается 108, а идентификатор заканчивается на 112

In [30]:
#Создание мешка
X_pick = np.zeros((features.shape[0], 112))

for i, match_id in enumerate(features.index):
    for p in list(range(5)):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [31]:
#Удаление категориальных признаков
del features['lobby_type']
del features['r1_hero']
del features['r2_hero']
del features['r3_hero']
del features['r4_hero']
del features['r5_hero']
del features['d1_hero']
del features['d2_hero']
del features['d3_hero']
del features['d4_hero']
del features['d5_hero']

In [32]:
Xscl_n = scale.fit_transform(features) #Масштабируем

In [34]:
#Расчет после удаления категориальных признаков
CVL_n = []
time_LR_n = []
for i in range(10):
    start_time = datetime.datetime.now()

    k = 10**(i-5)
    LR = LogisticRegression(C=k)
    CVL_n.append(np.mean(cross_val_score(estimator=LR, X=Xscl_n, y=y, cv=kf, scoring='roc_auc')))
    
    time_LR_n.append(datetime.datetime.now() - start_time)

In [35]:
C = 10**(CVL_n.index(max(CVL_n))-5)
T = time_LR_n[CVL_n.index(max(CVL_n))]
print('Score')
print(max(CVL_n))
print('C')
print(C)
print('time')
print(T)

Score
0.716559388563
C
0.01
time
0:00:49.962857


In [36]:
# Качество практически не изменилось, так как признаки были не сравнимы в том виде в котором они есть

In [37]:
#Добавление полученных признаков к числовым
data = np.concatenate([Xscl_n, X_pick], axis=1)

In [38]:
#Расчет после добавления
CVL_f = []
time_LR_f = []
for i in range(10):
    start_time = datetime.datetime.now()

    k = 10**(i-5)
    LR = LogisticRegression(C=k)
    CVL_f.append(np.mean(cross_val_score(estimator=LR, X=data, y=y, cv=kf, scoring='roc_auc')))
    
    time_LR_f.append(datetime.datetime.now() - start_time)

In [39]:
C = 10**(CVL_f.index(max(CVL_f))-5)
T = time_LR_f[CVL_f.index(max(CVL_f))]
print('Score')
print(max(CVL_f))
print('C')
print(C)
print('time')
print(T)

Score
0.75194757692
C
0.1
time
0:01:51.542379


In [40]:
#Качество оценки стало лучше, время затрачено больше. 
#Лучший парметр регуляризации С=0.1
#Качество улучшилось за счет увеличения характеристик которые являются значимыми

In [41]:
LR = LogisticRegression(C=0.1)
LR.fit(data, y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [42]:
features_test = features_test.fillna(0)

In [43]:
#Создание мешка для тестовой выборки
X_pick_test = np.zeros((features_test.shape[0], 112))

for i, match_id in enumerate(features_test.index):
    for p in list(range(5)):
        X_pick_test[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1


In [44]:
#Удаление категориальных признаков из тестовой выборки
del features_test['lobby_type']
del features_test['r1_hero']
del features_test['r2_hero']
del features_test['r3_hero']
del features_test['r4_hero']
del features_test['r5_hero']
del features_test['d1_hero']
del features_test['d2_hero']
del features_test['d3_hero']
del features_test['d4_hero']
del features_test['d5_hero']

In [45]:
Xscl_test = scale.fit_transform(features_test) #Масштабируем тестовую выборку

In [46]:
#Объединение
data_test = np.concatenate([Xscl_test, X_pick_test], axis=1)

In [47]:
prediction = LR.predict_proba(data_test)[:, 1]

In [48]:
#Максимальное значение прогноза
max(prediction)

0.99645865693077917

In [49]:
#Минимальное значение прогноза
min(prediction)

0.0085807263678184722